<a href="https://colab.research.google.com/github/islam12k/Home_Sales/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [69]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [70]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",",inferSchema=True,header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [71]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [72]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("sales")


In [73]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query="""
Select bedrooms,round(avg(price),2) as Average_market_price_bedroom_4
from sales
where bedrooms=4
group by bedrooms



"""
spark.sql(query).show()

+--------+------------------------------+
|bedrooms|Average_market_price_bedroom_4|
+--------+------------------------------+
|       4|                     299661.01|
+--------+------------------------------+



In [74]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query="""
Select date_built, bedrooms,bathrooms,round(avg(price),2) as Average_market_price_bedroom_3_bath_3
from sales
where bedrooms=3 and bathrooms=3
group by date_built,bedrooms,bathrooms



"""
spark.sql(query).show()

+----------+--------+---------+-------------------------------------+
|date_built|bedrooms|bathrooms|Average_market_price_bedroom_3_bath_3|
+----------+--------+---------+-------------------------------------+
|      2016|       3|        3|                            290555.07|
|      2012|       3|        3|                            293683.19|
|      2015|       3|        3|                             288770.3|
|      2010|       3|        3|                            292859.62|
|      2014|       3|        3|                            290852.27|
|      2013|       3|        3|                            295962.27|
|      2011|       3|        3|                            291117.47|
|      2017|       3|        3|                            292676.79|
+----------+--------+---------+-------------------------------------+



In [75]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query="""
Select date_built,bedrooms,bathrooms,floors,round(avg(price),2) as Average_market_price_bedroom_3_bath_3_with_two_floor
from sales
where bedrooms=3 and bathrooms=3 and floors=2
group by date_built, bedrooms,bathrooms,floors



"""
spark.sql(query).show()


+----------+--------+---------+------+----------------------------------------------------+
|date_built|bedrooms|bathrooms|floors|Average_market_price_bedroom_3_bath_3_with_two_floor|
+----------+--------+---------+------+----------------------------------------------------+
|      2012|       3|        3|     2|                                           295858.68|
|      2015|       3|        3|     2|                                           291788.36|
|      2014|       3|        3|     2|                                           294195.13|
|      2016|       3|        3|     2|                                            287812.6|
|      2011|       3|        3|     2|                                           281413.45|
|      2013|       3|        3|     2|                                           295142.13|
|      2010|       3|        3|     2|                                           280447.23|
|      2017|       3|        3|     2|                                          

In [76]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query="""
Select   view, round(avg(price),2) as average_price

from sales

where price>=350000
group by view



"""

spark.sql(query).show()




print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.506195068359375 seconds ---


In [77]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [78]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [79]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query="""
Select   view, round(avg(price),2) as average_price

from sales

where price>=350000
group by view



"""



spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.36502861976623535 seconds ---


In [80]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_per_year")

In [81]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet("home_sales_per_year")

In [82]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView("home_sales_p")

In [83]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query="""
Select   view, round(avg(price),2) as average_price

from sales

where price>=350000
group by view



"""

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- 0.24404692649841309 seconds ---


In [84]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")

DataFrame[]

In [85]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("sales")

False